#TF-IDF

##Random Forest and Logistic Regression

import of necessary libraries

In [ ]:
!pip install pymorphy2

In [ ]:
import pandas as pd

import nltk
nltk.download('punkt')
from string import punctuation
import re


import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

import of data

In [ ]:
new_df = pd.read_csv('data.csv')

In [ ]:
data_new = new_df.copy()

Train-Test

In [ ]:
y = data_new.x
x=data_new.drop('x',axis=1)
x_train, x_test, y_train, y_test =train_test_split(x,y,test_size=0.2)

preprosessing

In [ ]:
def prosessing(text):
    #remove html
    text = re.sub(r'\<[^>]*\>', '', text)
    #remove URLs
    text = re.sub(r"https?://[^,\s]+,?", "", text)
    #remove number
    text = re.sub(r"\d", "", text)
    text = re.sub('[^а-яА-Я]', ' ', text)
    # tokenization
    tokens = nltk.word_tokenize(text)
    #punctuation removal
    clean_words = [w.strip(punctuation) for w in tokens]
    #lowercase
    clean_words = [w.lower() for w in clean_words if w != '']
    #str
    f1 = ""
    for i in clean_words:
      f1 += str(i)+ " " 
    return f1

In [ ]:
def lemmatize(text):
    words = text.split() 
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [ ]:
x_train['text'] = x_train['text'].apply(lambda x: prosessing(x))
x_train['text'] = x_train['text'].apply(lambda x: lemmatize(x))

In [ ]:
x_test['text'] = x_test['text'].apply(lambda x: prosessing(x))
x_test['text'] = x_test['text'].apply(lambda x: lemmatize(x))

In [ ]:
x_train["text"] = x_train["text"].apply(lambda x: ' '.join(x))
x_test["text"] = x_test["text"].apply(lambda x: ' '.join(x))

x_train

tf-idf

In [ ]:
tfidf_vectorizer = TfidfVectorizer()


X_train = tfidf_vectorizer.fit_transform(x_train["text"])
Y_train = y_train
X_test = tfidf_vectorizer.transform(x_test["text"])

data balancing

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
X_train_miss, Y_train_miss = nm.fit_resample(X_train, Y_train)

In [ ]:
sns.countplot(x=Y_train_miss, data=data, palette='hls')
plt.show()
plt.savefig('count_plot')

In [ ]:
X_test_miss, Y_test_miss = nm.fit_resample(X_test_padded, y_test)

logistic regression

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=150).fit(X_train_miss, Y_train_miss)
predictions1 = clf.predict(X_test_miss)

In [ ]:
accuracy_score(Y_test_miss, predictions1)

In [ ]:
check = []
for element in Y_test_miss:
    check.append(element)

In [ ]:
target_names = ['0', '1']
print(classification_report(check, predictions1, target_names=target_names))

random forest

In [ ]:
rn = RandomForestClassifier(max_depth=2, random_state=0).fit(X_train_miss, Y_train_miss)
predictions1 = rn.predict(X_test_miss)

In [ ]:
accuracy_score(Y_test_miss, predictions1)

In [ ]:
check = []
for element in Y_test_miss:
    check.append(element)

In [ ]:
target_names = ['0', '1']
print(classification_report(check, predictions1, target_names=target_names))

##CNN and RNN

In [ ]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPool1D, Flatten, Dropout, LSTM, Conv1D
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split

import of data

In [ ]:
new_df = pd.read_csv('data.csv')

In [ ]:
data_new = new_df.copy()

In [ ]:
def prosessing(text):
    #remove html
    text = re.sub(r'\<[^>]*\>', '', text)
    #remove URLs
    text = re.sub(r"https?://[^,\s]+,?", "", text)
    #remove number
    text = re.sub(r"\d", "", text)
    text = re.sub('[^а-яА-Я]', ' ', text)
    # tokenization
    tokens = nltk.word_tokenize(text)
    #punctuation removal
    clean_words = [w.strip(punctuation) for w in tokens]
    #lowercase
    clean_words = [w.lower() for w in clean_words if w != '']
    #str
    f1 = ""
    for i in clean_words:
      f1 += str(i)+ " " 
    return f1

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


def lemmatize(text):
    words = text.split() 
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [ ]:
new_df["text"] = new_df["text"].apply(prosessing)

In [ ]:
new_df["text"] = new_df["text"].apply(lemmatize)

In [ ]:
new_df["text"] = new_df["text"].apply(lambda x: ' '.join(x))

In [ ]:
docs = np.asarray(new_df['text'])
labels = new_df['x'].to_numpy()

X_train, X_test , y_train, y_test = train_test_split(docs, labels , test_size = 0.2, stratify=labels, random_state=42)

vocabulary size

In [ ]:
token_counts = Counter()
for sent in X_train:
    token_counts.update(sent.split(' '))   
dict_size = len(token_counts.keys())
dict_size

In [ ]:
tokenizer = Tokenizer(num_words=dict_size)
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)

In [ ]:
max_comment_length = 200
X_train_padded = pad_sequences(X_train_tokenized, maxlen=max_comment_length)
X_test_padded = pad_sequences(X_test_tokenized, maxlen=max_comment_length)

Балансировка классов

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
X_train_miss, Y_train_miss = nm.fit_resample(X_train_padded, y_train)

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
X_test_miss, Y_test_miss = nm.fit_resample(X_test_padded, y_test)

CNN

In [ ]:
max_features = dict_size
embedding_dim = 64


model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=max_comment_length))
model.add(Conv1D(filters=embedding_dim*2, kernel_size=5, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

model.summary()

In [ ]:
epochs = 5

history = model.fit(X_train_miss, Y_train_miss, epochs=epochs, validation_data=(X_test_miss, Y_test_miss), batch_size=512)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'test'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test_miss,Y_test_miss)
print('Testing Accuracy is {} '.format(accuracy*100))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label = 'loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy & loss')
plt.ylim([0.0, 1])
plt.legend(loc='right')
plt.show()

In [ ]:
pred_y = model.predict(X_test_miss, verbose=1)
prediction = [1 if i >= .5 else 0 for i in pred_y]

In [ ]:
from sklearn.metrics import  classification_report
print(classification_report(Y_test_miss, prediction))

RNN (LSTM)

In [ ]:
max_features = dict_size
embedding_dim = 64


model2 = Sequential()
model2.add(Embedding(input_dim=max_features, output_dim=embedding_dim)) 
model2.add(LSTM(32, return_sequences=True))
model2.add(GlobalMaxPool1D())
model2.add(Dropout(0.3))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(1, activation = 'sigmoid'))
model2.compile(loss='BinaryCrossentropy', optimizer = 'adam', metrics = ['accuracy'])

model2.summary()

In [ ]:
epochs = 50

history2 = model2.fit(X_train_miss, Y_train_miss, epochs=epochs, validation_data=(X_test_miss, Y_test_miss), batch_size=512)

In [ ]:
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.legend(['train', 'test'], loc='upper right')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()
plt.show()

In [ ]:
loss, accuracy = model2.evaluate(X_test_miss,Y_test_miss)
print('Testing Accuracy is {} '.format(accuracy*100))

In [ ]:
plt.plot(history2.history['accuracy'], label='accuracy')
plt.plot(history2.history['loss'], label = 'loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy & loss')
plt.ylim([0.0, 1])
plt.legend(loc='right')
plt.show()

In [ ]:
pred_y = model2.predict(X_test_miss, verbose=1)
prediction = [1 if i >= .5 else 0 for i in pred_y]

In [ ]:
from sklearn.metrics import  classification_report
print(classification_report(Y_test_miss, prediction))

#Features

##Random Forest and Logistic Regression

feature extraction

In [ ]:
!pip install pymorphy2

import of necessary libraries

In [ ]:
import pandas as pd

import nltk
nltk.download('punkt')
from string import punctuation
import re


import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

import of data

In [ ]:
new_df = pd.read_csv('data.csv')

In [ ]:
data_new = new_df.copy()

preprosessing

In [ ]:
def prosessing(text):
    #remove html
    text = re.sub(r'\<[^>]*\>', '', text)
    #remove URLs
    text = re.sub(r"https?://[^,\s]+,?", "", text)
    #remove number
    text = re.sub(r"\d", "", text)
    text = re.sub('[^а-яА-Я]', ' ', text)
    # tokenization
    tokens = nltk.word_tokenize(text)
    #punctuation removal
    clean_words = [w.strip(punctuation) for w in tokens]
    #lowercase
    clean_words = [w.lower() for w in clean_words if w != '']
    #str
    f1 = ""
    for i in clean_words:
      f1 += str(i)+ " " 
    return f1

In [ ]:
def lemmatize(text):
    words = text.split() 
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

lexical diversity

In [ ]:
def counter(text):
    li = []
    for element in text:
        if element not in li:
            li.append(element)
    return len(li)

In [ ]:
data_new['lex_div'] = data_new['proses'].apply(lambda x: counter(x))

In [ ]:
count = []
for element in data_new['lex_div']:
    count.append(element)
    
l = []
for element in data_new['proses']:
    l.append(len(element))

In [ ]:
verbosity_list = []
for element in count:
    index = count.index(element)
    n = l[index]
    d = element/n
    verbosity_list.append(d)

In [ ]:
data_new = data_new.drop(columns='lex_div')

In [ ]:
data_new["lex_div"] = verbosity_list
data_new

average sentence length

In [ ]:
def preposessing_mean(text):
    #sentences
    sentences = nltk.sent_tokenize(text)
    #len
    sentence_word_length = [len(sent.split()) for sent in sentences]
    #mean len
    mean_sentence_len = np.mean(sentence_word_length)
    return mean_sentence_len

In [ ]:
data_new['word_sent'] = data_new['text'].apply(lambda x: preposessing_mean(x))

I - frequency

In [ ]:
i = []
for element in data_new['proses']:
  c =0
  for el in element:
    if el == 'я':
      c+=1
  res = c/len(element)
  i.append(res)

In [ ]:
data_new["i_freq"] = i
data_new

POS frequecy

In [ ]:
def pos(text):
    tags = nltk.pos_tag(text)
    return tags

In [ ]:
def count_freq_noun(text):
    counter = 0
    for element in text:
        if element[1] == 'NN' or element[1] == 'NNS' or element[1] == 'NNP' or element[1] == 'NNPS':
            counter +=1
    a = len(text)
    res = counter/a
    return res

def count_freq_adj(text):
    counter = 0
    for element in text:
        if element[1] == 'JJ' or element[1] == 'JJS' or element[1] == 'JJR':
            counter +=1
    res = counter/len(text)
    return res

def count_freq_prepos(text):
    counter = 0
    for element in text:
        if element[1] == 'IN':
            counter +=1
    res = counter/len(text)
    return res

def count_freq_pronoun(text):
    counter = 0
    for element in text:
        if element[1] == 'PRP' or element[1] == 'PRP$':
            counter +=1
    res = counter/len(text)
    return res

def count_freq_verb(text):
    counter = 0
    for element in text:
        if element[1] == 'VB' or element[1] == 'VBG' or element[1] == 'VBD' or element[1] == 'VBN' or element[1] == 'VBP' or element[1] == 'VBZ':
            counter +=1
    a = len(text)
    res = counter/a
    return res


In [ ]:
data_new['pos'] = data_new['proses'].apply(lambda x: pos(x))

In [ ]:
data_new['noun'] = data_new['pos'].apply(lambda x: count_freq_noun(x))
data_new['verb'] = data_new['pos'].apply(lambda x: count_freq_verb(x))
data_new['pronoun'] = data_new['pos'].apply(lambda x: count_freq_pronoun(x))
data_new['preposition'] = data_new['pos'].apply(lambda x: count_freq_prepos(x))
data_new['adjective'] = data_new['pos'].apply(lambda x: count_freq_adj(x))

readability

In [ ]:
! pip install ruts

In [ ]:
from ruts import ReadabilityStats
rs = ReadabilityStats(text)
redabl = []
for element in data_new['text']:
  rs = ReadabilityStats(element)
  r = rs.flesch_reading_easy
  redabl.append(r)

In [ ]:
data_new["redabl"] = redabl
data_new

sentiment

In [ ]:
!pip install dostoevsky

In [ ]:
!python -m dostoevsky download fasttext-social-network-model

In [ ]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [ ]:
messages = data_new['text']
results = model.predict(messages, k=2)
data_new['polarity'] = results

In [ ]:
el = 0
for element in data_new['text']:
  x = len(element.split(' '))
  el+=x

In [ ]:
f = []
for element in data_new['polarity']:
  x = max(element, key=element.get)
  f.append(x)

In [ ]:
data_new['sentiment'] = f
data_new

In [ ]:
data_new['y'] = pd.factorize(data_new['sentiment'])[0]

Correlation analysis of the dataset

In [ ]:
corr_matrix = data_new.corr()
corr_matrix

In [ ]:
corr_matrix["x"].sort_values(ascending=False)

Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()

In [ ]:
da = data_new.copy()

In [ ]:
da['lex_div'] = minmax.fit_transform(np.array(da['lex_div']).reshape(-1, 1))
da['word_sent'] = minmax.fit_transform(np.array(da['word_sent']).reshape(-1, 1))
da['i_freq'] = minmax.fit_transform(np.array(da['i_freq']).reshape(-1, 1))
da['preposition'] = minmax.fit_transform(np.array(da['preposition']).reshape(-1, 1))
da['pronoun'] = minmax.fit_transform(np.array(da['pronoun']).reshape(-1, 1))
da['verb'] = minmax.fit_transform(np.array(da['verb']).reshape(-1, 1))
da['noun'] = minmax.fit_transform(np.array(da['noun']).reshape(-1, 1))
da['y'] = minmax.fit_transform(np.array(da['y']).reshape(-1, 1))
da['adjective'] = minmax.fit_transform(np.array(da['adjective']).reshape(-1, 1))
da['redabl'] = minmax.fit_transform(np.array(da['redabl']).reshape(-1, 1))

Train-Test

In [ ]:
y = da.x
x=da.drop('x',axis=1)
x_train, x_test, y_train, y_test =train_test_split(x,y,test_size=0.2)

random forest

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0).fit(x_train, y_train)
predictions1 = clf.predict(x_test)

In [ ]:
check = []
for element in y_test:
    check.append(element)

In [ ]:
target_names = ['0', '1']
print(classification_report(check, predictions1, target_names=target_names))

logistic regression

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=150).fit(df_train, y_train)
predictions1 = clf.predict(df_test)

In [ ]:
check = []
for element in y_test:
    check.append(element)

In [ ]:
target_names = ['0', '1']
print(classification_report(check, predictions1, target_names=target_names))

## CNN and RNN

In [ ]:
import pandas as pd
from sklearn import preprocessing
res = preprocessing.normalize(da, axis=0)

In [ ]:
docs = res
labels = da['x'].to_numpy()

X_train, X_test , y_train, y_test = train_test_split(docs, labels , test_size = 0.2, stratify=labels, random_state=42)

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
X_train_miss, Y_train_miss = nm.fit_resample(X_train, y_train)

In [ ]:
from imblearn.under_sampling import NearMiss
nm = NearMiss()
X_test_miss, Y_test_miss = nm.fit_resample(X_test, y_test)

CNN

In [ ]:
max_features = 9
embedding_dim = 64


model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=max_comment_length))
model.add(Conv1D(filters=embedding_dim*2, kernel_size=5, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

model.summary()

In [ ]:
epochs = 5

history = model.fit(X_train_miss, Y_train_miss, epochs=epochs, validation_data=(X_test_miss, Y_test_miss), batch_size=512)

In [ ]:
pred_y = model.predict(X_test_miss, verbose=1)
prediction = [1 if i >= .5 else 0 for i in pred_y]

In [ ]:
from sklearn.metrics import  classification_report
print(classification_report(Y_test_miss, prediction))

RNN (LSTM)

In [ ]:
max_features = 9
embedding_dim = 64


model2 = Sequential()
model2.add(Embedding(input_dim=max_features, output_dim=embedding_dim)) 
model2.add(LSTM(32, return_sequences=True))
model2.add(GlobalMaxPool1D())
model2.add(Dropout(0.3))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(1, activation = 'sigmoid'))
model2.compile(loss='BinaryCrossentropy', optimizer = 'adam', metrics = ['accuracy'])

model2.summary()

In [ ]:
epochs = 50

history2 = model2.fit(X_train_miss, Y_train_miss, epochs=epochs, validation_data=(X_test_miss, Y_test_miss), batch_size=512)

In [ ]:
pred_y = model2.predict(X_test_miss, verbose=1)
prediction = [1 if i >= .5 else 0 for i in pred_y]

In [ ]:
from sklearn.metrics import  classification_report
print(classification_report(Y_test_miss, prediction))

#BERT

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score

In [ ]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [ ]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

loading a pre-trained model

In [ ]:
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased-sentence', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [ ]:
data_new = new_df.copy()

In [ ]:
# train and test
y = data_new.x
x=data_new.drop('x',axis=1)
x_train, x_test, y_train, y_test =train_test_split(x,y,test_size=0.2)

In [ ]:
train_text = x_train['text'].astype('str')

In [ ]:
train_labels = y_train

In [ ]:
test_text = x_test['text'].astype('str')

In [ ]:
test_labels = y_test

In [ ]:
seq_len_train = [len(str(i).split()) for i in x_train['text']]
seq_len_test = [len(str(i).split()) for i in x_test['text']]
max_seq_len = max(max(seq_len_test), max(seq_len_train))
max_seq_len

3195967

In [ ]:
max_seq_len = 512

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.values,
    max_length = max_seq_len,
    padding = 'max_length',
    truncation = True
)

In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train, train_labels)
test_dataset = Data(tokens_test, test_labels)

In [ ]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'F1': f1}

In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
training_args = TrainingArguments(
    output_dir = '/results', #Выходной каталог
    num_train_epochs = 3, #Кол-во эпох для обучения
    per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=21)

In [ ]:
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = train_dataset,
                  compute_metrics = compute_metrics)

In [ ]:
trainer.train()

In [ ]:
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

In [ ]:
print(classification_report(test_labels, pred))
print(f1_score(test_labels, pred))